In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import Aligner2, parallel_where

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon


In [2]:
available_label_indices = sorted(labels_sided_indices.values())

In [3]:
labelIndexMap_sidedToUnsided = {i: labels_unsided_indices[labelMap_sidedToUnsided[labels_sided[i-1]]] 
                                for i in available_label_indices}

In [4]:
labelIndexMap_m2f = {labels_sided_indices[name_s]: labels_unsided_indices[name_u] 
                        for name_s, name_u in labelMap_sidedToUnsided.iteritems() \
                        if name_s in [labels_sided[i-1] for i in available_label_indices]}
print labelIndexMap_m2f

{1: 1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5, 9: 5, 10: 6, 11: 7, 12: 7, 13: 8, 14: 8, 15: 9, 16: 9, 17: 10, 18: 10, 19: 11, 20: 11, 21: 12, 22: 13, 23: 13, 24: 14, 25: 14, 26: 15, 27: 16}


In [5]:
# atlas_volume = bp.unpack_ndarray_file(os.path.join(volume_dir, 'MD589/volume_MD589_annotation_withOuterContour.bp'))
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp')

In [6]:
all_indices_m = list(set(labelIndexMap_m2f.keys()) & set(np.unique(atlas_volume)))

In [7]:
all_indices_f = set([labelIndexMap_m2f[m] for m in all_indices_m])

In [9]:
from collections import defaultdict
results = defaultdict(dict)

In [10]:
# for i, stack in enumerate(['MD589', 'MD594', 'MD585', 'MD590', 'MD593', 'MD592', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']):
for i, stack in enumerate(['MD591']):
    
    subject_volume_allLabels = {}

    for ind_f in set(labelIndexMap_sidedToUnsided.values()):
        print ind_f

        subject_volume_roi = bp.unpack_ndarray_file(os.path.join(volume_dir, '%(stack)s/%(stack)s_scoreVolume_%(label)s.bp' % \
                                                          {'stack': stack, 'label': labels_unsided[ind_f-1]})).astype(np.float16)
        subject_volume_allLabels[ind_f] = subject_volume_roi
        del subject_volume_roi
        
    gradient_filepath_map_f = {ind_f: volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(label)s_%%(suffix)s.bp' % \
                           {'stack': stack, 'label': labels_unsided[ind_f-1]}
                           for ind_f in set(labelIndexMap_sidedToUnsided.values())}

    nzvoxels_m = Parallel(n_jobs=16)(delayed(parallel_where)(atlas_volume, i, num_samples=int(1e5))
                                     for i in all_indices_m)
    nzvoxels_m = dict(zip(all_indices_m, nzvoxels_m))
    
    aligner = Aligner2(volume_f_=subject_volume_allLabels, volume_m_=atlas_volume, 
                  nzvoxels_m_=nzvoxels_m,
                  labelIndexMap_m2f=labelIndexMap_m2f)

    aligner.set_centroid(centroid_m='volume_centroid',
                         centroid_f='volume_centroid')

    # aligner.load_gradient(gradient_filepath_map_f=gradient_filepath_map_f)

    # trial_combinations = [['7N_R', '7n_L'], ['7n_R', '7n_L'], ['12N', '7n_L', 'Pn_R']]
    trial_combinations = [[labels_sided[i-1] for i in all_indices_m]]

    aligner.load_gradient(gradient_filepath_map_f=gradient_filepath_map_f, 
                          indices_f=set([labels_unsided_indices[labelMap_sidedToUnsided[name_s]] 
                                         for name_s in set(chain(*trial_combinations))]))

    for names_sided_considered in trial_combinations:

    #     for _ in range(10):

        indices_sided_considered = [labels_sided_indices[name_s] for name_s in names_sided_considered]
    #     indices_unsided_considered = set([labels_unsided_indices[labelMap_sidedToUnsided[name_s]] 
    #                                       for name_s in names_sided_considered])

        t = time.time()
        T, scores = aligner.optimize(type='rigid', grid_search_iteration_number=3, grid_search_sample_number=5000,
                                 max_iter_num=5000, history_len=200, terminate_thresh=1e-3,
                                indices_m=indices_sided_considered)

    #     T, scores = aligner.optimize(type='rigid', init_T=T0.flatten(),
    #                              max_iter_num=5000, history_len=200, terminate_thresh=1e-3,
    #                             indices_m=indices_sided_considered)
        sys.stderr.write('Optimize: %f seconds\n' % (time.time() - t))

        score0 = aligner.compute_score(T.flatten())

    #     trials.append((score0, T)) 

        # sampling in gradient computation does not bring much variance: 
        # mean = 0.754, std = 0.002, # mean 0.729692610098 std 0.00407366793653

    #     H = None
    #     nbr_std = None

        t = time.time()
        H = aligner.compute_hessian(T, step=np.r_[1e-1, 1e-1, 1e-1, 10,
                                              1e-1, 1e-1, 1e-1, 10,
                                              1e-1, 1e-1, 1e-1, 10])
        sys.stderr.write('Compute hessian: %f seconds\n' % (time.time() - t))

        t = time.time()
        # when n = 1000, std is very stable; when n = 100, variation is +/- 0.1
        nbr_scores = aligner.compute_scores_neighborhood_random(T.flatten(), n=1000, stds=[0,0,0,10,0,0,0,10,0,0,0,10])
        nbr_std = np.std(nbr_scores)
        sys.stderr.write('Compute std: %f seconds\n' % (time.time() - t))


        _, invidual_scores = aligner.compute_score(T, return_individual_score=True)

        overlap_scores = {labels_sided[ind_s-1]: sc * 1e6 / len(nzvoxels_m[ind_s]) for ind_s, sc in invidual_scores.iteritems()}
    #     overlap_scores_sorted = sorted([(sc, name_s) for name_s, sc in overlap_scores.items()], reverse=True)

        results[stack][frozenset(names_sided_considered)] = (T, scores, score0, H, nbr_std, overlap_scores)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


load gradient 1: 6.254262 seconds
load gradient 2: 7.490490 seconds
load gradient 3: 8.068458 seconds
load gradient 4: 7.372368 seconds
load gradient 5: 6.148550 seconds
load gradient 6: 6.199468 seconds
load gradient 7: 6.065736 seconds
load gradient 8: 8.404813 seconds
load gradient 9: 10.426805 seconds
load gradient 10: 18.284049 seconds
load gradient 11: 32.882385 seconds
load gradient 12: 21.180019 seconds
load gradient 13: 26.481314 seconds
load gradient 14: 29.874030 seconds
overall: 195.138937 seconds
grid search: 72.143770 seconds
grid search: 54.583759 seconds
grid search: 42.090100 seconds
Optimize: 622.272324 seconds
Compute hessian: 48.158022 seconds
Compute std: 18.393220 seconds


In [51]:
# from registration_utilities import Aligner2, parallel_where

In [69]:
z_compiled = defaultdict(dict)

for stack, stack_data in results.iteritems():
    for names_s, (T, scores, score0, H, nbr_std, overlap_scores) in stack_data.iteritems():
        
        overlap_scores_sorted = sorted([(sc, name_s) for name_s, sc in overlap_scores.items()], reverse=True)
        print stack, '\n', overlap_scores_sorted, '\n'
        
        U, S, VT = np.linalg.svd(H[np.r_[3,7,11], np.r_[3,7,11][:,None]])
        z_principle = 1./2. * 10**2 * -S / nbr_std        
        z_compiled[stack][names_s] = np.r_[score0, z_principle]

MD602 
[(0.49679763615131378, '5N_R'), (0.4375331848859787, '7N_R'), (0.3559565544128418, 'Pn_R'), (0.32744184136390686, 'Pn_L'), (0.25930583477020264, '7n_R'), (0.22632801905274391, 'LRt_R'), (0.20196752622723579, 'Tz_R'), (0.17780011519789696, '5N_L'), (0.15729334205389023, '7n_L'), (0.15499207191169262, 'VLL_R'), (0.14231458306312561, 'RtTg'), (0.13434555381536484, 'VLL_L'), (0.12434851912933104, 'Amb_R'), (0.097973514348268509, '7N_L'), (0.037442747240073314, 'Amb_L'), (0.035934117312055522, 'LC_R'), (0.014998300466686487, 'LRt_L'), (0.0077662518405903986, 'LC_L'), (0.0042626549839042127, '12N'), (0.0035876299840681405, 'AP'), (0.0022404888295568526, 'Tz_L'), (0.00084953716912095371, 'R_L'), (0.0006498168382193971, 'R_R'), (0.00018851821466782354, '6N_L'), (0.0, '6N_R')] 

MD603 
[(0.42793277651071548, 'Pn_R'), (0.38246061652898788, 'RtTg'), (0.32789628952741623, 'Pn_L'), (0.31248394399881363, '7N_R'), (0.30993241816759109, '5N_R'), (0.29598390683531761, 'Tz_R'), (0.231331270188093